# Generating an INP Files with TOPAS

TOPAS Suite is a specialized software that will allow us to simulate the X-Ray Pattern of the compounds in the COD using a INP File. A INP uses the crystal system, space group symbol, lattice parameters(lenght and angles) and the Wyckoff's sties to generate accurately XRP representations. 

* In this step, we would like to do sample augmentation, due to the XRP changes depending on the crystal size and specimen displacement. We consider usual crystal sizes, such as: 200, 400, 600, 800, 1k nm, and usual specimen displacements such as -0.1, 0, 0.1.

Using this configuration for each CIF file we'll get over 15 XRP. 

## Libraries

In [1]:
import pandas as pd 
import os 

## Data Load

Define the Path to the Parquet file containing the organic or inorganic compounds:

In [2]:
df = pd.read_parquet('/home/bokhimi/COD/database/dataframes/df_raw_inorg.parquet')

In [3]:
df.head() # List of the five firsts elements

,cif,compound,cs,sg_number,sg_symbol,abc,angles,sites
0,1000171,Ti8Tl8W8O39.9984F8.0016,cubic,227,Fd-3m,"[10.241000, 10.241000, 10.241000]","[90.000000, 90.000000, 90.000000]","[[Tl+:0.250, 0.405, 0.405, 0.405, 32e], [Ti4+:..."
1,1000061,Al(HO)3,monoclinic,14,P2_1/c,"[5.062000, 8.671000, 4.713000]","[90.000000, 90.270000, 90.000000]","[[Al3+:1, 0.527, 0.167, 0.985, 4e], [H+:1, 0.3..."
2,1000062,SnO2,tetragonal,136,P4_2/mnm,"[4.738000, 4.738000, 3.186500]","[90.000000, 90.000000, 90.000000]","[[Sn4+:1, 0, 0, 0, 2a], [O2-:1, 0.3071, 0.3071..."
3,1000064,Fe2SiO4,orthorhombic,62,Pnma,"[4.819500, 10.478800, 6.087300]","[90.000000, 90.000000, 90.000000]","[[Fe2+:1, 0, 0, 0, 4a], [Fe2+:1, 0.98598, 0.28..."
4,1000005,Sr5V3H3(O3F8)2,monoclinic,14,P2_1/c,"[11.217000, 8.177500, 19.887000]","[90.000000, 105.999000, 90.000000]","[[Sr:1, 0.49726, 0.91552, 0.14705, 4e], [Sr:1,..."


In [4]:
df.shape # (Number of compounds, data of each compounds)

(45850, 8)

## TOPAS Simulation

For the simulation we employed TOPAS Suite V6. Specially the _tc_ software. The next function iterates over the dataframe, generating a INP file for each compunds using the column information. This INP file is given to _tc_ returning a _file.xy_ with the angle/intensity. The simulation window goes from 2 degree up to 135 degree with a step of 0.1.

In [ ]:
# Simulacion con TOPAS
'''
Este apartado se encarga de la creación de los archivos INP, es decir, a partir
del DataFrame genere el formato necesario de cada compuesto para ser ingresado
en TOPAS y obtener el .xy
'''

def create_input(df, sample):

    a = '''
    Para cada muestra en el df se crea un archivo input de TOPAS.
    Esta función se debe ejecutar simultaneamente con TOPAS,
    pues cada archivo .inp se sobreescribe por cada muestra.

    Input----> sample: muestra de un df (df.iloc[i])
    Output----> file.inp: archivo de entrada a TOPAS'''

    # --- Crea la carpeta para guardar los datos
    try:
        os.stat('samples')
    except:
        os.mkdir('samples')

    # Crea el archivo .inp

    if os.path.exists('samples\input_file.inp'): # Si existia, borra contenido
        file = open("samples\input_file.inp","r+")
        file.truncate(0)
        file.close()

    input_file = open('samples\\input_file.inp', 'w') # Abre en escritura



    #---------------------- Escritura del inp
    # Recupera los valores del df para cada compuesto
    numcif = df.iloc[sample]['cif']
    formula = df.iloc[sample]['compound']
    sg = df.iloc[sample]['sg_symbol']
    a = df.iloc[sample]['abc'][0]
    b = df.iloc[sample]['abc'][1]
    c = df.iloc[sample]['abc'][2]
    alpha = df.iloc[sample]['angles'][0]
    beta = df.iloc[sample]['angles'][1]
    gamma = df.iloc[sample]['angles'][2]

    # Escribe los valores en el .inp
    print(numcif, '\n') # Numero CIF de la muestra

    a = '''
    El analisis completo de esta funcion se encuentra en la Tesis, como resumen
    se le dan los parametros, las distintas medidas del cristal y la variacion
    de posicion, se le dan los valores min y maximos de simulacion
    junto con el paso. Se desprecia la radiación de fondo
    '''

    input_file.write(
                'iters 0' + '\n' + '\n' +
                'num_runs 15' + '\n' + '\n' +
                "'Varying average crystallite size 200, 400, 600, 800 y 1000 nm" + '\n' +
                "'Varying specimen displacement -0.01. 0.0, 0.01" + '\n' +
                'File_Variables(aver_size_0, 200, 1000, 200, sp_d, -0.01, 0.01, 0.01)' + '\n' + '\n' +
                'yobs_eqn = 1;' + '\n' +
                '   min 2.0' + '\n' +
                '   max 135.0' + '\n' +
                '   del 0.01' + '\n' + '\n' +
                'Out_X_Ycalc( ' + numcif + '-##Run_Number##.xy)' + '\n' + '\n' +
                'bkg 0 0 0 0' + '\n' + '\n' +
                'prm !spec_disp = #out sp_d ;' + '\n' + '\n' +
                'Specimen_Displacement (spec_disp)' + '\n' + '\n' +
                'str' + '\n' +
                'scale 0.01' + '\n' +
                'phase_name ' + formula + '\n' +
                'space_group ' + sg + '\n' +
                'a ' + str(a) + '\n' +
                'b ' + str(b) + '\n' +
                'c ' + str(c) + '\n' +
                'al ' + str(alpha) + '\n' +
                'be ' + str(beta) + '\n' +
                'ga ' + str(gamma) + '\n')

    # Se obtienen y escriben las coordenadas atomicas
    num = '0123456789'
    sites = df.iloc[sample]['sites']
    for site in sites:
        if all(map(str.isdigit, site[-1].replace('.', ''))):
            site = site[:-1]
        if len(site)==5:
            # site=[atom:occ, x, y, z, wyckoff]
            x = site[1]
            y = site[2]
            z = site[3]
            occ = site[0].split(':')[-1]
            multiplicity = site[4][:-1]

            atom = site[0].split(':')[0]
            atom =  re.sub(r'[0-9]+', '', atom).replace('.', '').replace('+', '').replace('-', '')

            input_file.write('site ' + atom + ' x ' + x + ' y '  + y + ' z ' + z +
                  ' occ '  + atom + ' ' + occ + ' beq 0.5' + '\n')

        else:
            # site=[atom_1:occ_1, ..., atom_n:occ_n, x, y, z, wyckoff]
            x = site[-4]
            y = site[-3]
            z = site[-2]
            multiplicity = site[-1][:-1]

            k = site[:-4]
            # k=[atom_1:occ_1, ..., atom_n:occ_n]
            for i in k:
                occ = i.split(':')[-1].replace(',', '')
                atom = i.split(':')[0]
                atom =  re.sub(r'[0-9]+', '', atom).replace('.', '').replace('+', '').replace('-', '')

                input_file.write('site ' + atom + ' x ' + x + ' y '  + y + ' z ' + z +
                      ' occ '  + atom + ' ' + occ + ' beq 0.5' + '\n')
    input_file.write(
                '\n' +

                'prm !aver_size  = #out aver_size_0 ;' + '\n' + '\n' +
                "lor_fwhm = 0.1 57.295779 * Lam /(Cos(Th)*aver_size) ;  'in nanometers" + '\n' + '\n' +
                ' prm  !strain 0.0001' + '\n' +
                '      gauss_fwhm = strain Tan(Th) ;' + '\n' + '\n' +
                   #"'X-ray Source Cu anode, Hoelzer(Phys. Rev. A 6, 4554 (1997)" + '\n' +
                    'lam'  + '\n' +
                    'ymin_on_ymax 0.00001' + '\n' +
                    '        la  0.579  lo  1.5405902 lh  0.4374157' + '\n' +
                    '        la  0.080  lo  1.5410632 lh  0.6432140' + '\n' +
                    '        la  0.236  lo  1.5443983 lh  0.5128764' + '\n' +
                    '        la  0.105  lo  1.5446848 lh  0.6872322' + '\n' +
                    '        la  0.0016 lo  1.3922   lh  0.55' + '\n' + '\n' +
                    '        Radius(217.5)' + '\n' + '\n' +
                #"' Lorentz Polarization corrections. No monochromator; then th2_monochromator = 0" + '\n' +
                'LP_Factor(!th2_monochromator, 0)'+ '\n' + '\n' +
                #"'Full axial divergence model from Coehlo TOPAS V 6" + '\n' +
                'Full_Axial_Model(12, 15, 16, 2.55, 3.10)'+ '\n' + '\n' +
                '/*' + '\n' +
                #"'Full axial divergence model from Coehlo TOPAS V 4" + '\n' +
                'filament_length               12' + '\n' +
                'sample_length                 15' + '\n' +
                "receiving_slit_length         16   'this is the detector slit" + '\n' +
                'primary_soller_angle       2.55' + '\n' +
                '    secondary_soller_angle   3.10' + '\n'  + '\n' +
                ' Divergence(, 0.5)' + '\n' +
                "    Slit_Width(, 0.075)         'This is the detector slit" + '\n' +
                '*/' + '\n'  + '\n' +
                #"'This is the macro Tube_Tails" + '\n' +
                'prm !b1  0.00381' + '\n' +
                'prm !c1  -0.9278' + '\n' +
                'prm !d1  1.94324' + '\n' +
                'prm !e1  0.00007' + '\n' + '\n' +
                '  stacked_hats_conv' + '\n' +
                '  whole_hat  = b1 57.2957795130823 / Rs;' + '\n' +
                '  half_hat   = c1 57.2957795130823 / Rs ;' + '\n' +
                '  hat_height = e1 ;' + '\n' +
                '  half_hat   = d1 57.2957795130823 / Rs ;' + '\n' +
                '  hat_height = e1 ;' + '\n' #+
                #"'End of the macro Tube_Tails"
    )

The following _for_ iterates over the whole dataframe: 

In [ ]:
for sample in range(len(df_raw)):
    create_input(df_raw, sample)
    !C:\Topas-6\tc C:\Users\Larec_4\Desktop\generacion-de-muestras-vf\samples\input_file.inp

    # Ruta del .exe de TOPAS (Windows) ---- # Ruta del archivo INP

    #print('\n')